In [47]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm
import multiprocessing as mp

me.mechanics_printing()

In [48]:
def Find_Fixed_Points_JEC(b_modes,t_modes,model_num,save_mini_model=False):
    dataset_name = 'JEC'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 12
    p.c.value = 1.8
    p.EI.value = 9.77e6
    p.GJ.value = 0.99e6
    p.rho_t.value = 19.53
    p.e_0.value = 0.08
    p.e_1.value = 0  
    p.ratio_fwt.value = 0.2

    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    p.k_fwt.value = 0    

    vars_ls =[]
    vars_ls.append((p.V,[70]))
    vars_ls.append((p.alpha_r,np.deg2rad([4])))
    vars_ls.append((p.m_factor,[1]))
    vars_ls.append((p.c_dmax,[1]))
    vars_ls.append((p.k_fwt,[0]))
    vars_ls.append((p.Lambda,np.deg2rad([17.5])))
    variables = [k for k,v in vars_ls]

    #Get The Model
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    #L = me.msubs((sm_mini.ExtForces.Q()[0]),{i:0 for i in p.qd})
    #L_func = sym.lambdify((p.q,variables),L)

    fp_df = rw.fixed_point_finder(p,sm,vars_ls,False)
    if save_mini_model:
        sm_mini = sm.msubs(p.GetSubs(0,None,ignore=variables+[p.w_g]))
        sm_mini.to_file(f'Temp_{dataset_name}_mini_M{model_num}_{b_modes}B{t_modes}T.py')
    fp_df['Model'] = model_num
    fp_df['Config'] = dataset_name
    return fp_df    

In [71]:
def Find_Fixed_Points_WT(b_modes,t_modes,model_num,save_mini_model=False):
    dataset_name = 'WT'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 1.345
    p.c.value = 0.15
    p.EI.value = 61
    p.GJ.value = 200
    p.rho_t.value = 10
    p.e_0.value = 0
    p.e_1.value = 0
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6  

    vars_ls =[]
    vars_ls.append((p.V,[35]))
    vars_ls.append((p.alpha_r,np.deg2rad([4])))
    vars_ls.append((p.m_factor,[1]))
    vars_ls.append((p.c_dmax,[1]))
    vars_ls.append((p.k_fwt,[0]))
    vars_ls.append((p.Lambda,np.deg2rad([17.5])))
    variables = [k for k,v in vars_ls]

    #Get The Model
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    #L = me.msubs((sm_mini.ExtForces.Q()[0]),{i:0 for i in p.qd})
    #L_func = sym.lambdify((p.q,variables),L)

    fp_df = rw.fixed_point_finder(p,sm,vars_ls,False)
    if save_mini_model:
        sm_mini = sm.msubs(p.GetSubs(0,None,ignore=variables+[p.w_g]))
        sm_mini.to_file(f'Temp_{dataset_name}_mini_M{model_num}_{b_modes}B{t_modes}T.py')
    fp_df['Model'] = model_num
    fp_df['Config'] = dataset_name
    return fp_df   

In [72]:
def Find_Fixed_Points_HALE(b_modes,t_modes,model_num,save_mini_model=False):
    dataset_name = 'HALE'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 0.0889
    p.s_t.value = 16
    p.c.value = 1
    p.EI.value = 2e4
    p.GJ.value = 1e4
    p.rho_t.value = 0.75
    p.e_0.value = 0.25
    p.e_1.value = 0
    
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6  

    vars_ls =[]
    vars_ls.append((p.V,[40]))
    vars_ls.append((p.alpha_r,np.deg2rad([4])))
    vars_ls.append((p.m_factor,[1]))
    vars_ls.append((p.c_dmax,[1]))
    vars_ls.append((p.k_fwt,[0]))
    vars_ls.append((p.Lambda,np.deg2rad([17.5])))
    variables = [k for k,v in vars_ls]

    #Get The Model
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    #L = me.msubs((sm_mini.ExtForces.Q()[0]),{i:0 for i in p.qd})
    #L_func = sym.lambdify((p.q,variables),L)

    fp_df = rw.fixed_point_finder(p,sm,vars_ls,False)
    if save_mini_model:      
        sm_mini = sm.msubs(p.GetSubs(0,None,ignore=variables+[p.w_g]))
        sm_mini.to_file(f'Temp_{dataset_name}_mini_M{model_num}_{b_modes}B{t_modes}T.py')
    fp_df['Model'] = model_num
    fp_df['Config'] = dataset_name
    return fp_df   

In [73]:
pool = mp.Pool(mp.cpu_count())

res =[]
for i in [0,3]:
    res.append(pool.apply_async(Find_Fixed_Points_JEC, args = (3,3,i,True)))
    res.append(pool.apply_async(Find_Fixed_Points_WT, args = (3,3,i,True)))
    res.append(pool.apply_async(Find_Fixed_Points_HALE, args = (3,3,i,True)))

pool.close()
pool.join()


ls = []
for i in res:
    ls += i.get().to_dict('records')
df = pd.DataFrame(ls)
print('Complete')

df['Coast Angle [Deg]'] = (np.rad2deg([i[-1] for i in df['q']]))
df.head()

M3-JEC: Started
M3-WT: Started
M0-HALE: Started
M0-WT: Started
M0-JEC: Started
M3-HALE: Started
Complete


,V,alpha_r,delta_m,c_dmax,k_fwt,Lambda,q,Model,Config,Coast Angle [Deg]
0,70.0,0.069813,1.0,1.0,0.0,0.305433,"[0.02524007020994529, 0.0066907963100454925, -...",0,JEC,-9.772270
1,35.0,0.069813,1.0,1.0,0.0,0.305433,"[3.973491305611813e-05, 0.18498408758617987, -...",0,WT,5.863527
2,40.0,0.069813,1.0,1.0,0.0,0.305433,"[0.0012408339807284926, 0.24435541011847622, -...",0,HALE,23.370913
3,70.0,0.069813,1.0,1.0,0.0,0.305433,"[0.02524007020994424, 0.006690796310044971, -0...",3,JEC,-9.772270
4,35.0,0.069813,1.0,1.0,0.0,0.305433,"[3.973491305619069e-05, 0.18498408758682072, -...",3,WT,5.863527


In [84]:
def GustTimeResponse(b_modes,t_modes,model_file,gust_strength,gust_period,fp_dict,att_index,end_time):
    print(f'Model {model_file},Attitude {att_index},Strength {gust_strength}, Period {gust_period} - Starting')
    p = rw.JEC_params(b_modes+t_modes+2)      
    sm = mf.SymbolicModel.from_file(model_file)   
    nm = sm.CreateNumericModel(p)
    
    # set attitude parameters
    tup = p.GetTuple()
    for k,v in fp_dict.items():
        for s in tup:
            if isinstance(s,mf.ModelSymbol):
                if s.name == k:
                    s.value = v

    #Define the Gust Function
    def gust(t,x):
        if t<gust_period:
            return gust_strength*0.5*(1- np.cos(2*np.pi*t/gust_period))
        else:
            return 0
    p.w_g.value = gust
    
    # Define Objective Function for solver
    def objectiveDeriv(t,y):
        tup = p.GetNumericTuple(y,t)
        return nm.deriv(t,y,tup)

    # Generate I.C. and run the solver
    x = [0]*p.qs*2
    x[::2] = fp_dict['q']
    print(x)
    yData = solve_ivp(objectiveDeriv,(0,end_time),x)

    #interpolate to reduce t and y points
    int_func = interp1d(yData.t,yData.y)

    t = np.linspace(0,end_time,(end_time*100)+1)
    yi = int_func(t)
    
    
    #Package Results
    # remove q from dict
    var = {k:v for k,v in fp_dict.items() if k !='q'}
    
    #generate functions for values of interest
    z,_ = mf.ShapeFunctions_BN_TM(b_modes,0,p.q[1:b_modes+1],p.y_0,0,0,0,p.eta[1:b_modes+1])

    d2z_dy2 = z.diff(p.y_0,p.y_0).subs(p.y_0,0)
    z_tip = z.subs(p.y_0,p.s_0)
    dz_dy_tip = z.diff(p.y_0).subs(p.y_0,p.s_0)

    d2z_dy2_f = sym.lambdify((p.GetTuple(),p.q),d2z_dy2)
    z_tip_f = sym.lambdify((p.GetTuple(),p.q),z_tip)
    dz_dy_tip_f = sym.lambdify((p.GetTuple(),p.q),dz_dy_tip)

    tup = p.GetNumericTuple(None,0)
    wrbm = d2z_dy2_f(tup,yi[::2])*p.EI.value
    wrbm_max = max(wrbm)-wrbm[0]
    wrbm_min = min(wrbm)-wrbm[0]
    inner_tip_deflection = z_tip_f(tup,yi[::2])
    fold_angle = np.rad2deg((yi[-2]))#+dz_dy_tip_f(tup,yi[::2]))
        
    ls = []
    for i in range(len(t)):
        lift = nm.ExtForces(p.GetNumericTuple(yi[:,i],t[i]),yi[:,i],t[i])[0,0]

        ls.append({'t':t[i],'q':yi[:,i],'Gust Strength':gust_strength,
                   'Gust Period':gust_period,**var,'Attitude':att_index,
                   'Delta WRBM':wrbm[i]-wrbm[0],
                   'WRBM Max':wrbm_max,'WRBM Min':wrbm_min,
                   'WRBM':wrbm[i],'Inner Tip Deflection':inner_tip_deflection[i],
                  'Fold Angle':fold_angle[i],'Lift':lift})
    return ls

In [62]:
%%time
i=0
row = df.iloc[i]
config = row['Config']
Model = row['Model']
ls = GustTimeResponse(3,3,f'Temp_{config}_mini_M{Model}_{3}B{3}T.py',1,1,row.to_dict(),i,4)

Model Temp_JEC_mini_M0_3B3T.py,Attitude 0,Strength 1, Period 1 - Starting
CPU times: user 9.16 s, sys: 40.5 ms, total: 9.2 s
Wall time: 9.2 s


In [85]:
data = df.copy()
#data = data[data['Model']==0]
data = data[data['Config']=='WT']
w_max = [0]
w_T = [1]
b_modes = 3
t_modes = 3

pool = mp.Pool(mp.cpu_count())

res =[]
for T in w_T:
    for M in w_max:
        for i in range(len(data)):
            row = data.iloc[i]
            config = row['Config']
            Model = row['Model']
            res.append(pool.apply_async(GustTimeResponse, args = (3,3,f'Temp_{config}_mini_M{Model}_{b_modes}B{t_modes}T.py',M,T,row.to_dict(),i,10)))

pool.close()
pool.join()


ls = []
for i in res:
    ls += i.get()
gust_df = pd.DataFrame(ls)
print('Complete')

Model Temp_WT_mini_M3_3B3T.py,Attitude 1,Strength 0, Period 1 - Starting
Model Temp_WT_mini_M0_3B3T.py,Attitude 0,Strength 0, Period 1 - Starting
[3.973491305619069e-05, 0, 0.18498408758682072, 0, -0.10950431966433043, 0, 0.02424468435702218, 0, 0.002961519521163964, 0, -0.001376170781210116, 0, 6.838250183296476e-17, 0, 0.10233784413436625, 0]
[3.973491305611813e-05, 0, 0.18498408758617987, 0, -0.10950431966413292, 0, 0.02424468435702274, 0, 0.002961519521163952, 0, -0.0013761707812100865, 0, 5.107467149618461e-17, 0, 0.10233784419360548, 0]
Complete


In [86]:
gust_df.to_pickle('LCOResponseData35.pkl')